In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os
from dotenv import load_dotenv
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.agents import Tool, AgentExecutor, ConversationalAgent
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [16]:
!pip install openai langchain faiss-cpu pandas pydantic pygame gtts SpeechRecognition pyaudio

In [9]:
import os
from typing import List, Dict, Optional, Any
from pydantic import BaseModel, ConfigDict
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from datetime import datetime
import json

class MenuItem(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    urun: str
    Fiyat: float
    ingredients: str
    category: str

class Order(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    items: List[Dict[str, any]] = []
    total: float = 0.0

class RestaurantRAG:
    def __init__(self, menu_path: str):
        """Initialize the RestaurantRAG system"""
        self.menu_path = menu_path
        self.menu_items = self.load_menu()
        self.vector_store = self.create_vector_store()
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True,
            output_key="answer",
            input_key="human_input"
        )
        self.current_order = Order()
        self.setup_rag_chain()
        # Başlangıçta siparisler.txt'yi sil
        if os.path.exists("siparisler.txt"):
            os.remove("siparisler.txt")

    def load_menu(self) -> List[MenuItem]:
        """Load menu items from CSV file"""
        df = pd.read_csv(self.menu_path, delimiter=';')
        menu_items = []
        for _, row in df.iterrows():
            menu_items.append(
                MenuItem(
                    urun=row['urun'],
                    Fiyat=float(row['Fiyat']),
                    ingredients=row['ingredients'],
                    category=row['category']
                )
            )
        return menu_items

    def create_vector_store(self):
        """Create FAISS vector store from menu items"""
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=400
        )
        
        menu_texts = []
        for item in self.menu_items:
            text = f"Category: {item.category}\nItem: {item.urun}\n"
            text += f"Price: {item.Fiyat} TL\nIngredients: {item.ingredients}\n"
            menu_texts.append(text)

        texts = text_splitter.create_documents(menu_texts)
        embeddings = OpenAIEmbeddings(model="text-embedding-ada-002",
        chunk_size=8000)
        vector_store = FAISS.from_documents(texts,
        embeddings,
        normalize_L2=True )
        return vector_store

    def get_menu_by_category(self, category: Optional[str] = None) -> str:
        """Get menu items by category or full menu"""
        menu_text = ""
        if category:
            items = [item for item in self.menu_items if item.category.lower() == category.lower()]
            if not items:
                return f"Bu kategoride ürün bulunamadı: {category}"
            menu_text = f"\n{category.upper()} MENÜSÜ:\n"
            for item in items:
                menu_text += f"• {item.urun} - İçindekiler: {item.ingredients}\n"
        else:
            # Group items by category
            categories = {}
            for item in self.menu_items:
                if item.category not in categories:
                    categories[item.category] = []
                categories[item.category].append(item)
            
            menu_text = "\nTAM MENÜ:\n"
            for category, items in categories.items():
                menu_text += f"\n{category.upper()}:\n"
                for item in items:
                    menu_text += f"• {item.urun} - İçindekiler: {item.ingredients}\n"
        
        return menu_text

    def setup_rag_chain(self):
        """Setup the RAG chain with improved order detection"""
        template = """You are a helpful and friendly Turkish restaurant waiter. Use the following information to help the customer.
    
        Menu Context:
        {context}
    
        Current Order Status:
        {current_order}
    
        Chat History:
        {chat_history}
    
        Customer: {question}
        {human_input}
    
        Instructions:
        YOUR NAME IS "MUTLU GARSON"
        1. When a customer places an order or modifies their existing order, you MUST respond with a JSON structure:
        {{"add_items": [{{"urun": "item_name", "quantity": number}}], "remove_items": [...], "response": "Your friendly response"}}
    
        2. For all other interactions (menu questions, general chat, etc.), respond normally in Turkish without JSON.
    
        Order Detection Guidelines:
        - Detect if the message contains a food/drink order
        - Orders can be in any format: casual requests, formal orders, questions about getting items
        - If you detect an order, ALWAYS respond with the JSON structure
        - Include quantities if specified, default to 1 if not specified
        - For order modifications, include both add_items and remove_items as needed
    
        Example Responses:
        For orders:
        {{"add_items": [{{"urun": "Pizza", "quantity": 2}}, {{"urun": "Cola", "quantity": 1}}], "remove_items": [], "response": "2 pizza ve 1 cola siparişinizi aldım. Başka bir arzunuz var mı?"}}
    
        For regular queries:
        "Menümüzde pizza çeşitlerimiz şunlardır..."
    
        Assistant:"""
    
        PROMPT = PromptTemplate(
            template=template,
            input_variables=["context", "question", "chat_history", "current_order", "human_input"]
        )
    
        self.qa_chain = ConversationalRetrievalChain.from_llm(
            llm=ChatOpenAI(temperature=0.2, model="gpt-4", max_tokens=2000),
            retriever=self.vector_store.as_retriever(
                search_type="mmr",
                search_kwargs={
                    "k": 30,
                    "fetch_k": 40,
                    "lambda_mult": 0.7
                }
            ),
            memory=self.memory,
            combine_docs_chain_kwargs={"prompt": PROMPT},
            return_source_documents=True,
            chain_type="stuff",
            verbose=True
        )

 


    def _try_json_parsing(self, response_text: str) -> bool:
        """Try to parse JSON response for order actions"""
        try:
            if isinstance(response_text, str) and "{" in response_text:
                actions = json.loads(response_text)
                if "add_items" in actions:
                    for item in actions["add_items"]:
                        self.add_to_order(item["urun"], item["quantity"])
                if "remove_items" in actions:
                    for item in actions["remove_items"]:
                        self.remove_from_order(item["urun"], item.get("quantity"))
                return True
            return False
        except json.JSONDecodeError:
            return False

    def _parse_text_for_orders(self, text: str) -> None:
        """Parse plain text response for order items"""
        try:
            # Initialize items if needed
            if not hasattr(self.current_order, 'items'):
                self.current_order.items = []
                self.current_order.total = 0.0
    
            # Look for common patterns indicating orders
            lines = text.split('\n')
            for line in lines:
                if line.strip().startswith('-') or line.strip().startswith('•'):
                    # Extract item name
                    item_text = line.strip('- •').strip()
                    
                    # Look for quantity indicators
                    quantity = 1
                    if 'x' in item_text:
                        parts = item_text.split('x')
                        try:
                            quantity = int(parts[0].strip())
                            item_text = parts[1].strip()
                        except ValueError:
                            pass
    
                    # Find matching menu item
                    menu_item = next(
                        (item for item in self.menu_items 
                         if item.urun.lower() in item_text.lower()),
                        None
                    )
                    
                    if menu_item:
                        self.add_to_order(menu_item.urun, quantity)
    
        except Exception as e:
            print(f"Text parsing error: {str(e)}")

    def add_to_order(self, item_name: str, quantity: int = 1) -> str:
        """Add item to current order with enhanced initialization"""
        try:
            # Ensure current_order is properly initialized
            if not hasattr(self.current_order, 'items'):
                self.current_order.items = []
                self.current_order.total = 0.0
    
            # Find menu item
            menu_item = next(
                (item for item in self.menu_items if item.urun.lower() == item_name.lower()),
                None
            )
            
            if not menu_item:
                return f"Üzgünüm, {item_name} menümüzde bulunamadı."
    
            # Calculate new item total
            new_total = menu_item.Fiyat * quantity
            
            # Check for existing item
            existing_item = next(
                (item for item in self.current_order.items 
                 if item["urun"].lower() == item_name.lower()),
                None
            )
    
            if existing_item:
                existing_item["quantity"] += quantity
                existing_item["total"] = existing_item["Fiyat"] * existing_item["quantity"]
            else:
                # Add new item
                order_item = {
                    "urun": menu_item.urun,
                    "quantity": quantity,
                    "Fiyat": menu_item.Fiyat,
                    "total": new_total
                }
                self.current_order.items.append(order_item)

            # Update total
            self.current_order.total = sum(item["total"] for item in self.current_order.items)
            
            print(f"Added to order: {quantity}x {menu_item.urun}")
            print(f"Current order: {self.get_order_summary()}")
            
            return f"{quantity}x {menu_item.urun} siparişinize eklendi. Toplam tutar: {self.current_order.total:.2f} TL"
    
        except Exception as e:
            print(f"Error in add_to_order: {str(e)}")
            return f"Sipariş eklenirken bir hata oluştu: {str(e)}"

    
    

    
            


    def remove_from_order(self, item_name: str, quantity: Optional[int] = None) -> str:
        """Remove item from current order with improved error handling"""
        try:
            # Siparişte ürünü bul
            item_index = next(
                (idx for idx, item in enumerate(self.current_order.items)
                 if item["urun"].lower() == item_name.lower()),
                None
            )

            if item_index is None:
                return f"{item_name} mevcut siparişinizde bulunmuyor."

            item = self.current_order.items[item_index]
            
            if quantity is None or quantity >= item["quantity"]:
                # Ürünü tamamen çıkar
                self.current_order.total -= item["total"]
                self.current_order.items.pop(item_index)
                return f"{item['urun']} siparişinizden çıkarıldı."
            else:
                # Miktarı azalt
                item["quantity"] -= quantity
                old_total = item["total"]
                item["total"] = item["Fiyat"] * item["quantity"]
                self.current_order.total -= (old_total - item["total"])
                return f"{quantity}x {item['urun']} siparişinizden çıkarıldı."

        except Exception as e:
            print(f"Sipariş çıkarma hatası: {str(e)}")
            return f"Sipariş çıkarılırken bir hata oluştu: {str(e)}"

    def get_order_summary(self) -> str:
        """Get current order summary with improved formatting"""
        try:
            if not self.current_order.items:
                return "Henüz sipariş verilmedi."
            
            summary = "\n=== SİPARİŞ ÖZETİ ===\n"
            for item in self.current_order.items:
                summary += f"• {item['urun']} x {item['quantity']} = {item['total']:.2f} TL\n"
            summary += "=" * 20 + "\n"
            summary += f"TOPLAM TUTAR: {self.current_order.total:.2f} TL\n"
            summary += "=" * 20
            return summary

        except Exception as e:
            print(f"Sipariş özeti oluşturma hatası: {str(e)}")
            return "Sipariş özeti oluşturulurken bir hata oluştu."


    def save_order_to_file(self, order_details: str):
        """Save order details to siparisler.txt"""
        with open("siparisler.txt", "w", encoding="utf-8") as f:
            f.write("=== Sipariş Detayları ===\n")
            f.write(f"Tarih: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(order_details)
            f.write("\n=== Sipariş Sonu ===")

    def format_final_order(self) -> str:
        """Format the final order details"""
        if not self.current_order.items:
            return "Sipariş boş."
        
        order_text = "\nSipariş Özeti:\n"
        order_text += "=" * 30 + "\n"
        for item in self.current_order.items:
            order_text += f"• {item['urun']} x{item['quantity']} = {item['total']:.2f} TL\n"
        order_text += "=" * 30 + "\n"
        order_text += f"Toplam Tutar: {self.current_order.total:.2f} TL"
        return order_text
        
    def process_message(self, message: str) -> str:
        """Process customer message using model-based order detection"""
        try:
            print(f"\nGelen mesaj: {message}")
            print("Mevcut sipariş durumu:")
            print(self.get_order_summary())
    
            inputs = {
                "question": message,
                "current_order": self.get_order_summary(),
                "chat_history": self.memory.load_memory_variables({})["chat_history"],
                "human_input": message
            }
    
            response = self.qa_chain(inputs)
            print(f"\nModel yanıtı: {response['answer']}")
    
            # JSON yanıt varsa siparişi işle
            if self._try_json_parsing(response["answer"]):
                current_order = self.get_order_summary()
                return f"{json.loads(response['answer'])['response']}\n\nGüncel Sipariş Durumu:\n{current_order}"
            else:
                return response['answer']
    
        except Exception as e:
            print(f"Mesaj işleme hatası: {str(e)}")
            return f"Mesajınız işlenirken bir hata oluştu: {str(e)}"
    
        



    def run(self):
        """Run the interactive session"""
        print("Restaurant Assistant: Hoş geldiniz! Size nasıl yardımcı olabilirim?")
        print("\nMenü Kategorileri:")
        categories = set(item.category for item in self.menu_items)
        for category in categories:
            print(f"- {category}")
        
        while True:
            
            try:
                user_input = input("\nSiz: ").strip()
                if user_input.lower() in ["exit", "quit", "bye", "çıkış", "görüşürüz"]:
                    if self.current_order.items:
                        final_order = self.format_final_order()
                        print("\nSiparişinizi onaylıyor musunuz?")
                        print(final_order)
                        confirmation = input("\nSiparişi onaylıyor musunuz? (evet/hayır): ").strip().lower()
                        
                        if confirmation in ["evet", "e", "yes", "y"]:
                            self.save_order_to_file(final_order)
                            print("\nSiparişiniz onaylandı ve siparisler.txt dosyasına kaydedildi.")
                        else:
                            print("\nSipariş iptal edildi.")
                    
                    print("\nBizi tercih ettiğiniz için teşekkürler! İyi günler!")
                    break
                    
                response = self.process_message(user_input)
                print(f"\nAssistant: {response}")
                
            except Exception as e:
                print(f"\nBir hata oluştu: {str(e)}")

    def process_single_query(self, query: str) -> Dict[str, Any]:
        """
        Tek bir sorguyu işler ve sonuçları döndürür.
        Bu metod run() metodunun yanında alternatif olarak kullanılabilir.
        """
        try:
            inputs = {
                "question": query,
                "current_order": self.get_order_summary(),
                "chat_history": self.memory.load_memory_variables({})["chat_history"],
                "human_input": ""
            }
            
            response = self.qa_chain(inputs)
            
            try:
                actions = json.loads(response["answer"])
                if "add_items" in actions:
                    for item in actions["add_items"]:
                        self.add_to_order(item["urun"], item["quantity"])
                if "remove_items" in actions:
                    for item in actions["remove_items"]:
                        self.remove_from_order(item["urun"], item.get("quantity"))
                return {
                    "query": query,
                    "response": actions.get("response", "Order updated successfully."),
                    "order_status": self.get_order_summary(),
                    "success": True
                }
            except json.JSONDecodeError:
                return {
                    "query": query,
                    "response": response["answer"],
                    "order_status": self.get_order_summary(),
                    "success": True
                }
            
        except Exception as e:
            return {
                "query": query,
                "response": f"Error: {str(e)}",
                "order_status": self.get_order_summary(),
                "success": False
            }

    def reset_memory(self):
        """
        Sohbet geçmişini ve mevcut siparişi sıfırlar.
        Test senaryoları için kullanışlıdır.
        """
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True,
            output_key="answer",
            input_key="human_input"
        )
        self.current_order = Order()



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:547: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [19]:
def evaluate_performance(metrics: Dict[str, Any]):
    """
    Test sonuçlarını analiz eden fonksiyon
    """
    success_rate = (metrics["successful_queries"] / metrics["total_queries"]) * 100
    avg_response_time = metrics["total_time"] / metrics["total_queries"]
    
    print("\nDetailed Performance Analysis:")
    print(f"Success rate: {success_rate:.2f}%")
    print(f"Average response time: {avg_response_time:.2f} seconds")
    
    print("\nQuery-wise Analysis:")
    for response in metrics["responses"]:
        print(f"\nQuery: {response['query']}")
        print(f"Success: {'Yes' if response['success'] else 'No'}")
        print(f"Response time: {response['time_taken']:.2f} seconds")


In [8]:
# sonsuz döngü 
if __name__ == "__main__":
    # RestaurantRAG sistemini başlatın
    restaurant = RestaurantRAG("menu1.csv")
    restaurant.run()

Restaurant Assistant: Hoş geldiniz! Size nasıl yardımcı olabilirim?

Menü Kategorileri:
- Corbalar
- Hamburgerler
- Baslangiclar
- Pizzalar
- Ana Yemekler
- Tatlilar



Siz:  3 adet sufle alabilir miyim 



Gelen mesaj: 3 adet sufle alabilir miyim
Mevcut sipariş durumu:
Henüz sipariş verilmedi.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful and friendly Turkish restaurant waiter. Use the following information to help the customer.
    
        Menu Context:
        Category: Tatlilar
Item: Sufle
Price: 65.0 TL
Ingredients: Belcika cikolatasi, tereyagi, un, yumurta

Category: Baslangiclar
Item: Sebzeli Spring Roll
Price: 70.0 TL
Ingredients: Havuc, kabak, lahana, pirinc yufkasi, tatli eksi sos

Category: Ana Yemekler
Item: Antrikot
Price: 190.0 TL
Ingredients: ozel marine edilmis antrikot, kozlenmis sebzeler, patates puresi

Category: Ana Yemekler
Item: Sebzeli Guvec
Price: 85.0 TL
Ingredients: Firinlanmis taze sebzeler, domates sos

Category: Hamburgerler
Item: Siyah Burger
Price: 95.0 TL
Ingredients: Siyah fasulye koftesi, krem peynir, marul, sogan, domates, tursu

Category: Corbalar
Item: Tavuk Suyu corbasi
Pr


Siz:  çorba çeşitleriniz nelerdir acaba 



Gelen mesaj: çorba çeşitleriniz nelerdir acaba
Mevcut sipariş durumu:

=== SİPARİŞ ÖZETİ ===
• Sufle x 3 = 195.00 TL
TOPLAM TUTAR: 195.00 TL


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: 3 adet sufle alabilir miyim
Assistant: {"add_items": [{"urun": "Sufle", "quantity": 3}], "remove_items": [], "response": "3 adet Sufle siparişinizi aldım. Başka bir arzunuz var mı?"}
Follow Up Input: çorba çeşitleriniz nelerdir acaba
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful and friendly Turkish restaurant waiter. Use the following information to help the customer.
    
        Menu Context:
        Category: Corbalar
Item: Tavuk Suyu corbasi
Price: 50.0 TL
Ingredients: Tavuk suyu, haslanmis tavuk, arp

...


Gelen mesaj: ah 1 adet sufleden vazgeçtim onun yerine tiramisu alabilir miiym
Mevcut sipariş durumu:

=== SİPARİŞ ÖZETİ ===
• Sufle x 3 = 195.00 TL
• Balkabagi corbasi x 2 = 110.00 TL
TOPLAM TUTAR: 305.00 TL


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: 3 adet sufle alabilir miyim
Assistant: {"add_items": [{"urun": "Sufle", "quantity": 3}], "remove_items": [], "response": "3 adet Sufle siparişinizi aldım. Başka bir arzunuz var mı?"}
Human: çorba çeşitleriniz nelerdir acaba
Assistant: Çorbalar menümüzde aşağıdaki çeşitler bulunmaktadır:

1. Tavuk Suyu Çorbası - 50.0 TL
   İçerik: Tavuk suyu, haşlanmış tavuk, arpa şehriye, havuç, patates, kereviz, maydanoz

2. Balkabağı Çorbası - 55.0 TL
   İçerik: Balkabağı, soğan, sarımsak, krema, zeytinyağı, tavuk suyu, muskat, tuz, karabiber

3. Mercimek Çorbası - 40.0 


Siz:  bye



Siparişinizi onaylıyor musunuz?

Sipariş Özeti:
• Sufle x2 = 130.00 TL
• Balkabagi corbasi x2 = 110.00 TL
• Tiramisu x1 = 85.00 TL
Toplam Tutar: 325.00 TL



Siparişi onaylıyor musunuz? (evet/hayır):  evet



Siparişiniz onaylandı ve siparisler.txt dosyasına kaydedildi.

Bizi tercih ettiğiniz için teşekkürler! İyi günler!


In [ ]:
import time

In [ ]:
if __name__ == "__main__":
    metrics = test_restaurant_rag()
    evaluate_performance(metrics)


In [ ]:
import os
from typing import List, Dict, Optional
from pydantic import BaseModel, ConfigDict
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from datetime import datetime
import json
import speech_recognition as sr
import pyaudio
import wave
import threading
import time
from gtts import gTTS
import pygame
from io import BytesIO

class MenuItem(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    urun: str
    Fiyat: float
    ingredients: str
    category: str

class Order(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    items: List[Dict[str, any]] = []
    total: float = 0.0

class AudioRecorder:
    def __init__(self):
        self.is_recording = False
        self.frames = []
        self.silence_threshold = 500  # Sessizlik eşiği (milisaniye)
        self.last_sound = time.time()

    def start_recording(self):
        self.is_recording = True
        self.frames = []
        audio = pyaudio.PyAudio()
        stream = audio.open(format=pyaudio.paInt16,
                          channels=1,
                          rate=44100,
                          input=True,
                          frames_per_buffer=1024)

        print("Dinliyorum...")

        while self.is_recording:
            data = stream.read(1024)
            self.frames.append(data)
            
            # Ses seviyesini kontrol et
            audio_data = [int.from_bytes(data[i:i+2], byteorder='little', signed=True) 
                         for i in range(0, len(data), 2)]
            if max(map(abs, audio_data)) > 500:  # Ses seviyesi eşiği
                self.last_sound = time.time()
            
            # 5 saniye sessizlik kontrolü
            if time.time() - self.last_sound > 5:
                self.is_recording = False

        stream.stop_stream()
        stream.close()
        audio.terminate()

        # Kaydedilen sesi geçici bir WAV dosyasına kaydet
        with wave.open("temp.wav", 'wb') as wf:
            wf.setnchannels(1)
            wf.setsampwidth(2)
            wf.setframerate(44100)
            wf.writeframes(b''.join(self.frames))

class RestaurantRAG:
    def __init__(self, menu_path: str):
        """Initialize the RestaurantRAG system"""
        self.menu_path = menu_path
        self.menu_items = self.load_menu()
        self.vector_store = self.create_vector_store()
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True,
            output_key="answer",
            input_key="human_input"
        )
        self.current_order = Order()
        self.setup_rag_chain()
        self.recorder = AudioRecorder()
        pygame.mixer.init()
        
        # Başlangıçta siparisler.txt'yi sil
        if os.path.exists("siparisler.txt"):
            os.remove("siparisler.txt")

    def speech_to_text(self):
        """Convert speech to text using speech recognition"""
        recognizer = sr.Recognizer()
        with sr.AudioFile('temp.wav') as source:
            audio = recognizer.record(source)
            try:
                text = recognizer.recognize_google(audio, language='tr-TR')
                return text
            except sr.UnknownValueError:
                return ""
            except sr.RequestError:
                return "Ses tanıma servisi şu anda kullanılamıyor."

    def text_to_speech(self, text: str):
        """Convert text to speech and play it"""
        tts = gTTS(text=text, lang='tr')
        fp = BytesIO()
        tts.write_to_fp(fp)
        fp.seek(0)
        
        with open('temp.mp3', 'wb') as f:
            f.write(fp.read())
        
        pygame.mixer.music.load('temp.mp3')
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10)
        
        os.remove('temp.mp3')

    # [Sizin orijinal metodlarınız aynen korunuyor]
    def load_menu(self) -> List[MenuItem]:
        """Load menu items from CSV file"""
        df = pd.read_csv(self.menu_path, delimiter=';')
        menu_items = []
        for _, row in df.iterrows():
            menu_items.append(
                MenuItem(
                    urun=row['urun'],
                    Fiyat=float(row['Fiyat']),
                    ingredients=row['ingredients'],
                    category=row['category']
                )
            )
        return menu_items

    def create_vector_store(self):
        """Create FAISS vector store from menu items"""
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )
        
        menu_texts = []
        for item in self.menu_items:
            text = f"Category: {item.category}\nItem: {item.urun}\n"
            text += f"Price: {item.Fiyat} TL\nIngredients: {item.ingredients}\n"
            menu_texts.append(text)

        texts = text_splitter.create_documents(menu_texts)
        embeddings = OpenAIEmbeddings()
        vector_store = FAISS.from_documents(texts, embeddings)
        return vector_store

    def setup_rag_chain(self):
        """Setup the RAG chain with custom prompt"""
        # [Sizin orijinal prompt template'iniz ve chain kurulumu]
        template = """You are a helpful and friendly Turkish restaurant waiter. Use the following information to help the customer.

        Menu Context:
        {context}

        Current Order Status:
        {current_order}

        Chat History:
        {chat_history}

        Customer: {question}
        {human_input}

   Instructions:
   YOUR NAME IS "MUTLU GARSON"
   Avoid providing incomplete information when answering. Never skip any items from the menu when the customer asks for it.
   If the customer asks for the menu, provide the full list of items (including all dish names and descriptions). Only include prices if the customer specifically requests them.
	1.	Help customers understand the menu and answer their questions naturally.
	2.	Provide the full menu when explicitly asked. Do not include prices unless specifically requested.
	3.	List menu items within a given price range when the customer provides one.
	4.	If asked about an unavailable item, inform the customer politely that it is not on the menu and suggest the closest available alternative.
	5.	Answer ingredient-related and pricing questions accurately when prompted.
	6.	Track and confirm the current order to avoid mistakes.
	7.	Handle special requests or modifications professionally, respecting the menu limitations.
	8.	Maintain a friendly and professional tone throughout the interaction.
	9.	Always respond in Turkish.

   Examples of behavior:
	•	When asked for the menu: Provide the full list of items without prices unless specifically requested.
	•	When asked for items in a price range: List all menu items that match the given range, mentioning their prices.
	•	When an unavailable item is requested: "Bu ürün menümüzde yok, ancak [en yakın alternatif ürün] öneririm."


        If the customer wants to add or remove items, respond with a JSON structure:
        {{"add_items": [{{"urun": "item_name", "quantity": number}}], "remove_items": [...], "response": "Your friendly response"}}

        For regular queries about menu or general questions, just respond normally in Turkish.

        Assistant:"""

        PROMPT = PromptTemplate(
            template=template,
            input_variables=["context", "question", "chat_history", "current_order", "human_input"]
        )

        self.qa_chain = ConversationalRetrievalChain.from_llm(
            llm=ChatOpenAI(temperature=0.2, model="gpt-4"),
            retriever=self.vector_store.as_retriever(
                search_type="similarity",
                search_kwargs={"k": 5}
            ),
            memory=self.memory,
            combine_docs_chain_kwargs={"prompt": PROMPT},
            return_source_documents=True,
            chain_type="stuff",
            verbose=True
        )

    def add_to_order(self, item_name: str, quantity: int = 1) -> str:
        """Add item to current order"""
        for menu_item in self.menu_items:
            if menu_item.urun.lower() == item_name.lower():
                order_item = {
                    "urun": menu_item.urun,
                    "quantity": quantity,
                    "Fiyat": menu_item.Fiyat,
                    "total": menu_item.Fiyat * quantity
                }
                self.current_order.items.append(order_item)
                self.current_order.total += order_item["total"]
                return f"Added {quantity}x {menu_item.urun} to your order."
        return f"Sorry, couldn't find {item_name} in our menu."

    def remove_from_order(self, item_name: str, quantity: int = None) -> str:
        """Remove item from current order"""
        for idx, item in enumerate(self.current_order.items):
            if item["urun"].lower() == item_name.lower():
                if quantity is None or quantity >= item["quantity"]:
                    self.current_order.total -= item["total"]
                    self.current_order.items.pop(idx)
                    return f"Removed {item['urun']} from your order."
                else:
                    item["quantity"] -= quantity
                    item["total"] = item["quantity"] * item["Fiyat"]
                    self.current_order.total -= (quantity * item["Fiyat"])
                    return f"Removed {quantity}x {item['urun']} from your order."
        return f"Couldn't find {item_name} in your current order."

    def get_order_summary(self) -> str:
        """Get current order summary"""
        if not self.current_order.items:
            return "Your order is empty."
        
        summary = "\nCurrent Order:\n"
        for item in self.current_order.items:
            summary += f"• {item['urun']} x{item['quantity']} = {item['total']:.2f} TL\n"
        summary += f"\nTotal: {self.current_order.total:.2f} TL"
        return summary

    def save_order_to_file(self, order_details: str):
        """Save order details to siparisler.txt"""
        with open("siparisler.txt", "w", encoding="utf-8") as f:
            f.write("=== Sipariş Detayları ===\n")
            f.write(f"Tarih: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(order_details)
            f.write("\n=== Sipariş Sonu ===")

    def format_final_order(self) -> str:
        """Format the final order details"""
        if not self.current_order.items:
            return "Sipariş boş."
        
        order_text = "\nSipariş Özeti:\n"
        order_text += "=" * 30 + "\n"
        for item in self.current_order.items:
            order_text += f"• {item['urun']} x{item['quantity']} = {item['total']:.2f} TL\n"
        order_text += "=" * 30 + "\n"
        order_text += f"Toplam Tutar: {self.current_order.total:.2f} TL"
        return order_text

    def process_message(self, message: str) -> str:
        """Process customer message and return response"""
        try:
            inputs = {
                "question": message,
                "current_order": self.get_order_summary(),
                "chat_history": self.memory.load_memory_variables({})["chat_history"],
                "human_input": ""
            }
            
            response = self.qa_chain(inputs)
            
            try:
                actions = json.loads(response["answer"])
                if "add_items" in actions:
                    for item in actions["add_items"]:
                        self.add_to_order(item["urun"], item["quantity"])
                if "remove_items" in actions:
                    for item in actions["remove_items"]:
                        self.remove_from_order(item["urun"], item.get("quantity"))
                return actions.get("response", "Order updated successfully.")
            except json.JSONDecodeError:
                return response["answer"]
            
        except Exception as e:
            return f"Sorry, I encountered an error: {str(e)}"

    def run(self):
        """Run the interactive session"""
        welcome_message = "Hoş geldiniz! Size nasıl yardımcı olabilirim?"
        print(f"Restaurant Assistant: {welcome_message}")
        self.text_to_speech(welcome_message)

        print("\nMenü Kategorileri:")
        categories = set(item.category for item in self.menu_items)
        for category in categories:
            print(f"- {category}")
        
        while True:
            try:
                print("\nDinliyorum...")
                # Ses kaydını başlat
                recording_thread = threading.Thread(target=self.recorder.start_recording)
                recording_thread.start()
                recording_thread.join()
                
                # Sesi metne çevir
                user_input = self.speech_to_text()
                if not user_input:
                    continue
                    
                print(f"\nSiz: {user_input}")
                
                if user_input.lower() in ["exit", "quit", "bye", "çıkış", "görüşürüz"]:
                    if self.current_order.items:
                        final_order = self.format_final_order()
                        print("\nSiparişinizi onaylıyor musunuz?")
                        print(final_order)
                        self.text_to_speech("Siparişinizi onaylıyor musunuz?")
                        
                        recording_thread = threading.Thread(target=self.recorder.start_recording)
                        recording_thread.start()
                        recording_thread.join()
                        confirmation = self.speech_to_text().lower()
                        
                        if confirmation in ["evet", "e", "yes", "y", "tamam"]:
                            self.save_order_to_file(final_order)
                            print("\nSiparişiniz onaylandı ve siparisler.txt dosyasına kaydedild")
                            print("\n" + final_message)
                            self.text_to_speech(final_message)
                        else:
                            cancel_message = "Sipariş iptal edildi. Başka bir şey ister misiniz?"
                            print("\n" + cancel_message)
                            self.text_to_speech(cancel_message)
        
                    else:
                        goodbye = "Görüşmek üzere, iyi günler!"
                        print("\n" + goodbye)
                        self.text_to_speech(goodbye)
                    return

                response = self.process_message(user_input)
                print(f"\nRestaurant Assistant: {response}")
                self.text_to_speech(response) 
            except Exception as e:  # Hataları yakalayıp bildirir
                print(f"Hata oluştu: {e}")
                self.text_to_speech("Bir hata oluştu. Lütfen tekrar deneyin.")

In [ ]:
 if __name__ == "__main__":
    restaurant = RestaurantRAG("menu1.csv")
    try:
        restaurant.run()
    except KeyboardInterrupt:
        print("\nProgram sonlandırılıyor...")
    except Exception as e:
        print(f"\nBir hata oluştu: {str(e)}")
    finally:
        if os.path.exists("temp.wav"):
            os.remove("temp.wav")


